***GENERATED CODE FOR consumertrends PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '3120', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Male', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Item Purchased', 'transformation_label': 'String Indexer'}], 'feature': 'Item Purchased', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3120', 'mean': '', 'stddev': '', 'min': 'Backpack', 'max': 'T-shirt', 'missing': '0', 'distinct': '25'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Item Purchased'}, {'feature_label': 'Item Purchased', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Item Purchased')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Category', 'transformation_label': 'String Indexer'}], 'feature': 'Category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3120', 'mean': '', 'stddev': '', 'min': 'Accessories', 'max': 'Outerwear', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Category'}, {'feature_label': 'Category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Location', 'transformation_label': 'String Indexer'}], 'feature': 'Location', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3120', 'mean': '', 'stddev': '', 'min': 'Alabama', 'max': 'Wyoming', 'missing': '0', 'distinct': '50'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Location'}, {'feature_label': 'Location', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Location')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Size', 'transformation_label': 'String Indexer'}], 'feature': 'Size', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3120', 'mean': '', 'stddev': '', 'min': 'L', 'max': 'XL', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Size'}, {'feature_label': 'Size', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Size')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Color', 'transformation_label': 'String Indexer'}], 'feature': 'Color', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3120', 'mean': '', 'stddev': '', 'min': 'Beige', 'max': 'Yellow', 'missing': '0', 'distinct': '25'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Color'}, {'feature_label': 'Color', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Color')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Season', 'transformation_label': 'String Indexer'}], 'feature': 'Season', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3120', 'mean': '', 'stddev': '', 'min': 'Fall', 'max': 'Winter', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Season'}, {'feature_label': 'Season', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Season')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Subscription Status', 'transformation_label': 'String Indexer'}], 'feature': 'Subscription Status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3120', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Subscription Status'}, {'feature_label': 'Subscription Status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Subscription Status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Payment Method', 'transformation_label': 'String Indexer'}], 'feature': 'Payment Method', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3120', 'mean': '', 'stddev': '', 'min': 'Bank Transfer', 'max': 'Venmo', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Payment Method'}, {'feature_label': 'Payment Method', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Payment Method')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Shipping Type', 'transformation_label': 'String Indexer'}], 'feature': 'Shipping Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3120', 'mean': '', 'stddev': '', 'min': '2-Day Shipping', 'max': 'Store Pickup', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Shipping Type'}, {'feature_label': 'Shipping Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Shipping Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Discount Applied', 'transformation_label': 'String Indexer'}], 'feature': 'Discount Applied', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3120', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Discount Applied'}, {'feature_label': 'Discount Applied', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Discount Applied')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Promo Code Used', 'transformation_label': 'String Indexer'}], 'feature': 'Promo Code Used', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3120', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Promo Code Used'}, {'feature_label': 'Promo Code Used', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Promo Code Used')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Preferred Payment Method', 'transformation_label': 'String Indexer'}], 'feature': 'Preferred Payment Method', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3120', 'mean': '', 'stddev': '', 'min': 'Bank Transfer', 'max': 'Venmo', 'missing': '0', 'distinct': '6'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Preferred Payment Method'}, {'feature_label': 'Preferred Payment Method', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop(
            'Preferred Payment Method')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Frequency of Purchases', 'transformation_label': 'String Indexer'}], 'feature': 'Frequency of Purchases', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '3120', 'mean': '', 'stddev': '', 'min': 'Annually', 'max': 'Weekly', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Frequency of Purchases'}, {'feature_label': 'Frequency of Purchases', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Frequency of Purchases')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run consumertrendsHooks.ipynb
try:
	#sourcePreExecutionHook()

	shoppingtrends = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/shopping_trends.csv', 'filename': 'shopping_trends.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'is_header': 'Use Header Line', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://ml.colaberry.com:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run consumertrendsHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(shoppingtrends,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Customer ID", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3120", "mean": "1560.5", "stddev": "900.81", "min": "1", "max": "3120", "missing": "0"}, "updatedLabel": "Customer ID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3120", "mean": "44.1", "stddev": "15.3", "min": "18", "max": "70", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3120", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"feature_label": "Item Purchased", "transformation_label": "String Indexer"}], "feature": "Item Purchased", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3120", "mean": "", "stddev": "", "min": "Backpack", "max": "T-shirt", "missing": "0", "distinct": "25"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Item Purchased"}, {"transformationsData": [{"feature_label": "Category", "transformation_label": "String Indexer"}], "feature": "Category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3120", "mean": "", "stddev": "", "min": "Accessories", "max": "Outerwear", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Category"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Purchase Amount (USD)", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3120", "mean": "59.66", "stddev": "23.74", "min": "20", "max": "100", "missing": "0"}, "updatedLabel": "Purchase Amount (USD)"}, {"transformationsData": [{"feature_label": "Location", "transformation_label": "String Indexer"}], "feature": "Location", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3120", "mean": "", "stddev": "", "min": "Alabama", "max": "Wyoming", "missing": "0", "distinct": "50"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Location"}, {"transformationsData": [{"feature_label": "Size", "transformation_label": "String Indexer"}], "feature": "Size", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3120", "mean": "", "stddev": "", "min": "L", "max": "XL", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Size"}, {"transformationsData": [{"feature_label": "Color", "transformation_label": "String Indexer"}], "feature": "Color", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3120", "mean": "", "stddev": "", "min": "Beige", "max": "Yellow", "missing": "0", "distinct": "25"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Color"}, {"transformationsData": [{"feature_label": "Season", "transformation_label": "String Indexer"}], "feature": "Season", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3120", "mean": "", "stddev": "", "min": "Fall", "max": "Winter", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Season"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Review Rating", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "3120", "mean": "3.26", "stddev": "0.76", "min": "2.0", "max": "5.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Review Rating"}, {"transformationsData": [{"feature_label": "Subscription Status", "transformation_label": "String Indexer"}], "feature": "Subscription Status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3120", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Subscription Status"}, {"transformationsData": [{"feature_label": "Payment Method", "transformation_label": "String Indexer"}], "feature": "Payment Method", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3120", "mean": "", "stddev": "", "min": "Bank Transfer", "max": "Venmo", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Payment Method"}, {"transformationsData": [{"feature_label": "Shipping Type", "transformation_label": "String Indexer"}], "feature": "Shipping Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3120", "mean": "", "stddev": "", "min": "2-Day Shipping", "max": "Store Pickup", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Shipping Type"}, {"transformationsData": [{"feature_label": "Discount Applied", "transformation_label": "String Indexer"}], "feature": "Discount Applied", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3120", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Discount Applied"}, {"transformationsData": [{"feature_label": "Promo Code Used", "transformation_label": "String Indexer"}], "feature": "Promo Code Used", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3120", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Promo Code Used"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Previous Purchases", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "3120", "mean": "25.47", "stddev": "14.42", "min": "1", "max": "50", "missing": "0"}, "updatedLabel": "Previous Purchases"}, {"transformationsData": [{"feature_label": "Preferred Payment Method", "transformation_label": "String Indexer"}], "feature": "Preferred Payment Method", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3120", "mean": "", "stddev": "", "min": "Bank Transfer", "max": "Venmo", "missing": "0", "distinct": "6"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Preferred Payment Method"}, {"transformationsData": [{"feature_label": "Frequency of Purchases", "transformation_label": "String Indexer"}], "feature": "Frequency of Purchases", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "3120", "mean": "", "stddev": "", "min": "Annually", "max": "Weekly", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Frequency of Purchases"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run consumertrendsHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["Customer ID", "Age", "Purchase Amount (USD)", "Review Rating", "Previous Purchases", "Gender_stringindexer", "Item Purchased_stringindexer", "Category_stringindexer", "Size_stringindexer", "Color_stringindexer", "Season_stringindexer", "Subscription Status_stringindexer", "Payment Method_stringindexer", "Shipping Type_stringindexer", "Discount Applied_stringindexer", "Promo Code Used_stringindexer", "Preferred Payment Method_stringindexer", "Frequency of Purchases_stringindexer"], "Location_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

